In [ ]:
#Creating the image class to ease the operations
class Image:
    def __init__(self,image):
        self.image=image
    #calculate grayscale
    def thresh_hold(self):
        pass
    def to_gray_scale(self):
        pass